## Custom LLM

根据每个Action或Role配置不同的LLM。

- 定义配置：使用默认配置，或者从~/.metagpt目录中加载自定义配置。
- 分配配置：将特定的LLM配置分配给Role和Action。配置的优先级：Action config > Role config > Global config（config in config2.yaml）。
- 团队交互：创建一个带有环境的团队，开始交互。

```python
# 定义配置
from metagpt.config2 import Config

# 以下是一些示例配置，分别为gpt-4、gpt-4-turbo 和 gpt-3.5-turbo。
gpt4 = Config.from_home("gpt-4.yaml")  # 从`~/.metagpt`目录加载自定义配置`gpt-4.yaml`
gpt4t = Config.default()  # 使用默认配置，即`config2.yaml`文件中的配置，此处`config2.yaml`文件中的model为"gpt-4-turbo"
gpt35 = Config.default()
gpt35.llm.model = "gpt-3.5-turbo"  # 将model修改为"gpt-3.5-turbo"


# 为Role和Action分配配置
# 对于关注的Action而言，配置的优先级为：Action config > Role config > Global config 。
from metagpt.roles import Role
from metagpt.actions import Action

# 创建a1、a2和a3三个Action。并为a1指定`gpt4t`的配置。
a1 = Action(config=gpt4t, name="Say", instruction="Say your opinion with emotion and don't repeat it")
a2 = Action(name="Say", instruction="Say your opinion with emotion and don't repeat it")
a3 = Action(name="Vote", instruction="Vote for the candidate, and say why you vote for him/her")

# 创建A，B，C三个角色，分别为“民主党候选人”、“共和党候选人”和“选民”。
# 虽然A设置了config为gpt4，但因为a1已经配置了Action config，所以A将使用model为gpt4的配置，而a1将使用model为gpt4t的配置。
A = Role(name="A", profile="Democratic candidate", goal="Win the election", actions=[a1], watch=[a2], config=gpt4)
# 因为B设置了config为gpt35，而为a2未设置Action config，所以B和a2将使用Role config，即model为gpt35的配置。
B = Role(name="B", profile="Republican candidate", goal="Win the election", actions=[a2], watch=[a1], config=gpt35)
# 因为C未设置config，而a3也未设置config，所以C和a3将使用Global config，即model为gpt4t的配置。
C = Role(name="C", profile="Voter", goal="Vote for the candidate", actions=[a3], watch=[a1, a2])



# 交互
import asyncio
from metagpt.environment import Environment
from metagpt.team import Team

# 创建一个描述为“美国大选现场直播”的环境
env = Environment(desc="US election live broadcast")
team = Team(investment=10.0, env=env, roles=[A, B, C])
# 运行团队，我们应该会看到它们之间的协作
await team.run(idea="Topic: climate change. Under 80 words per message.", send_to="A", n_round=3) 

## Communication

智能体之间的消息交换是通过Message中提供标签的属性，以及Environment提供的publish_message能力来实现的。

- 智能体作为消息的发送者，只需提供消息的来源信息即可。消息的来源对应Message的sent_from、cause_by。
- 智能体作为消息的使用者，需要订阅相应的消息。消息的订阅标签对应Message的cause_by。
- Environment对象负责将消息按各个智能体的订阅需求，广播给各个智能体。

In [ ]:
class Message(BaseModel):
    id: str = Field(default="", validate_default=True)  # According to Section 2.2.3.1.1 of RFC 135
    content: str
    instruct_content: Optional[BaseModel] = Field(default=None, validate_default=True)
    role: str = "user"  # system / user / assistant
    cause_by: str = Field(default="", validate_default=True)
    sent_from: str = Field(default="", validate_default=True)
    send_to: set[str] = Field(default={MESSAGE_ROUTE_TO_ALL}, validate_default=True)

在规划智能体之间的消息转发流程时，首先要确定智能体的功能边界，这跟设计一个函数的套路一样：

- 智能体输入什么。智能体的输入决定了智能体对象的rc.watch的值。
- 智能体输出什么。智能体的输出决定了智能体输出Message的参数。
- 智能体要完成什么工作。智能体要完成什么工作决定了智能体有多少action，action之间按什么状态流转。

需求：

- 智能体 A 接受需求，分为 10 个子任务。
- 智能体 B 被分配执行 10 个子任务。
- 智能体 C 被分配编译 10 个子任务。
- 智能体 D 检查编译，向代理 B 提供反馈。

步骤 2-4 需要发生 3-4 次。如何构建我的系统以正确的方式实现？

In [ ]:
# 第一步：让Agent A能接收用户发过来的需求：

class AgentA(Role):
    def __init__(self, **kwargs) -> None:
        super().__init__(**kwargs)
        # Initialize actions specific to the Architect role
        self.set_actions([AgentAAction]) # 由于智能体只有1种action，所以不用改写_think函数。

        # 订阅消息
        self._watch({UserRequirement}) # UserRequirement是Message缺省的cause_by的值

# 第二步： AgentAAction中，将用户需求拆分成10份
class AgentAAction(Aciton):
    async def run(self, with_messages:List[Message]=None, **kwargs) -> List[str]:
        subtasks: List[str] = split_10_subtask(with_messages[0].content)
        return subtasks

# 第三步：Agent A在_act中run AgentAAction，然后将结果发给Agent B：
class AgentA(Role):
    async def _act(self) -> Message:
        subtasks = await self.rc.todo.run(self.rc.history)
        for i in subtasks:
           self.rc.env.publish_message(Message(content=i, cause_by=AgentAAction)) # 发送10条这种消息，Agent B订阅了这种类型的消息
        return Message(content="dummy message", send_to=MESSAGE_ROUTE_TO_NONE) # 消息已发，所以return一个空消息就行


# 第四步：处理BCD
class AgentBAction(Action):
        async def run(self, with_messages:List[Message]=None, **kwargs) -> Message:
            ... # 将结果填到Message里

class AgentB(Role):
    def __init__(self, **kwargs) -> None:
        super().__init__(**kwargs)
        # Initialize actions specific to the Architect role
        self.set_actions([AgentBAction]) # 由于智能体只有1种action，所以不用改写_think函数。

        # 订阅消息
        self._watch({AgentAAction, AgentDAction})



class AgentCAction(Action):
        async def run(self, with_messages:List[Message]=None, **kwargs) -> Message:
            ... # 将结果填到Message里

class AgentC(Role):
    def __init__(self, **kwargs) -> None:
        super().__init__(**kwargs)
        # Initialize actions specific to the Architect role
        self.set_actions([AgentCAction]) # 由于智能体只有1种action，所以不用改写_think函数。

        # 订阅消息
        self._watch({AgentBAction})


class AgentDAction(Action):
        async def run(self, with_messages:List[Message]=None, **kwargs) -> Message:
            ... # 将结果填到Message里

class AgentD(Role):
    def __init__(self, **kwargs) -> None:
        super().__init__(**kwargs)
        # Initialize actions specific to the Architect role
        self.set_actions([AgentDAction]) # 由于智能体只有1种action，所以不用改写_think函数。

        # 订阅消息
        self._watch({AgentCAction})


# 第五步：放入env
    context = Context() # Load config2.yaml
    env = Environment(context=context)
    env.add_roles([AgentA(), AgentB(), AgentC(), AgentD()])
    env.publish_message(Message(content='New user requirements', send_to=AgentA)) # 将用户的消息发送个Agent A，让Agent A开始工作。
    while not env.is_idle: # env.is_idle要等到所有Agent都没有任何新消息要处理后才会为True
        await env.run()

## Continously Develop

即在已有项目上继续开发。

`metagpt 需求 --project-path 项目路径`



示例：


创建游戏：`metagpt "基于pygame写一个命令行的贪吃蛇游戏" --project-name "snake_game" --run-tests --n-round 20 --max-auto-summarize-code 1`

新需求：`metagpt "添加一个随机出现的敌人，持续 5 秒。如果敌人被吃掉，游戏结束。如果敌人没有被吃掉，它会在 5 秒后消失。" --project-path "/Users/Yam/llm_course/MetaGPT/workspace/snake_game" --run-tests --n-round 20 --max-auto-summarize-code 1`

修复错误：`metagpt "TypeError: draw() takes 1 positional argument but 2 were given" --project-path "/Users/Yam/llm_courseMetaGPT/workspace/snake_game" --run-tests --n-round 10 --max-auto-summarize-code 0`


---

`metagpt "TypeError: Board.place_food() missing 1 required positional argument: 'snake'" --project-path /Users/Yam/Yam/llm_course/metagpt/MetaGPT/workspace/snake_game  --run-tests --n-round 10 --max-auto-summarize-code 0`

不支持：

- 架构相关的需求，比如setup.py或者requirements.txt怎么写。
- 指定特定函数应执行特定操作。

原因：

- 此类“要求”本质上是约束或指示。与真正的需求不同，它们指定系统内某些模块的功能，而不是整个系统的功能。
- MetaGPT Software公司将软件开发过程简化为类视图和序列图的需求，然后直接翻译成源代码。诸如上述的系统内部模块的设计要求，由于缺乏明确的实现流程，因此被放弃。


还可以通过直接编辑软件公司生成的项目文件来添加新的需求。 对相关文件进行必要的修改后，使用“metagpt”命令启动新一轮的增量迭代。



![](can_edit.png)

## Checkpoint


在程序运行过程中，记录程序不同模块的产出并落盘。当程序碰到外部如Ctrl-C或内部执行异常如LLM Api网络异常导致退出等情况时。再次执行程序，能够从中断前的结果中恢复继续执行，而无需从0到1开始执行，降低开发者的时间和费用成本。


- 序列化的操作根据不同模块的功能有所区分，比如角色基本信息，初始化后即可以进行序列化，过程中不会发生改变。记忆信息，需要执行过程中，实时进行序列化保证完整性。
- 这里统一在发生异常或正常结束运行时进行序列化。

可能中断情况：

- **网络**等问题，LLM-Api调用重试多次后仍失败
- Action执行过程中，输出内容**解析失败**导致退出
- 人为的Ctrl-C对程序进行中断（实际不支持）


序列化结构：

```json
{
  "env": {
    "desc": "",
    "roles": {
      "Role A": {},
      "Role B": {
        "name": "RoleB",
        "profile": "Role B",
        "goal": "RoleB's goal",
        "constraints": "RoleB's constraints",
        "desc": "",
        "is_human": false,
        "role_id": "",
        "states": [
          "0. <class 'tests.metagpt.serialize_deserialize.test_serdeser_base.ActionOK'>",
          "1. <class 'tests.metagpt.serialize_deserialize.test_serdeser_base.ActionRaise'>"
        ],
        "actions": [
          {
            "name": "ActionOK",
            "context": "",
            "prefix": "You are a Role B, named RoleB, your goal is RoleB's goal. the constraint is RoleB's constraints. ",
            "desc": "",
            "__module_class_name": "tests.metagpt.serialize_deserialize.test_serdeser_base.ActionOK"
          },
          {
            "name": "ActionRaise",
            "context": "",
            "prefix": "You are a Role B, named RoleB, your goal is RoleB's goal. the constraint is RoleB's constraints. ",
            "desc": "",
            "__module_class_name": "tests.metagpt.serialize_deserialize.test_serdeser_base.ActionRaise"
          }
        ],
        "rc": {
          "memory": {
            "storage": [
              {
                "id": "7cc01798c3324c6c8b676d282ea9e92c",
                "content": "ActionPass run passed",
                "instruct_content": {
                  "class": "pass",
                  "mapping": {
                    "result": "(<class 'str'>, Ellipsis)"
                  },
                  "value": {
                    "result": "pass result"
                  }
                },
                "role": "RoleA(Role A)",
                "cause_by": "tests.metagpt.serialize_deserialize.test_serdeser_base.ActionPass",
                "sent_from": "tests.metagpt.serialize_deserialize.test_serdeser_base.RoleA",
                "send_to": ["<all>"]
              }
            ],
            "index": {
              "tests.metagpt.serialize_deserialize.test_serdeser_base.ActionOK": [
                {
                  "id": "018bde1d4bdb4e9387c1053da0dc0cb3",
                  "content": "ok",
                  "instruct_content": null,
                  "role": "Role B",
                  "cause_by": "tests.metagpt.serialize_deserialize.test_serdeser_base.ActionOK",
                  "sent_from": "tests.metagpt.serialize_deserialize.test_serdeser_base.RoleB",
                  "send_to": ["<all>"]
                }
              ]
            },
            "ignore_id": false
          },
          "state": 1,
          "watch": [
            "tests.metagpt.serialize_deserialize.test_serdeser_base.ActionPass"
          ],
          "react_mode": "by_order",
          "max_react_loop": 1
        },
        "addresses": [
          "RoleB",
          "tests.metagpt.serialize_deserialize.test_serdeser_base.RoleB"
        ],
        "recovered": true,
        "latest_observed_msg": {
          "id": "7cc01798c3324c6c8b676d282ea9e92c",
          "content": "ActionPass run passed",
          "instruct_content": {
            "class": "pass",
            "mapping": {
              "result": "(<class 'str'>, Ellipsis)"
            },
            "value": {
              "result": "pass result"
            }
          },
          "role": "RoleA(Role A)",
          "cause_by": "tests.metagpt.serialize_deserialize.test_serdeser_base.ActionPass",
          "sent_from": "tests.metagpt.serialize_deserialize.test_serdeser_base.RoleA",
          "send_to": ["<all>"]
        },
        "__module_class_name": "tests.metagpt.serialize_deserialize.test_serdeser_base.RoleB"
      }
    },
    "history": "\nHuman: write a snake game\nRoleA(Role A): {'result': 'pass result'}\nHuman: write a snake game"
  },
  "investment": 10.0,
  "idea": "write a snake game"
}
```

恢复顺序：


- 情况1：角色A（1个action）-> 角色B（2个action），角色A进行action选择时出现异常退出。
- 情况2：角色A（1个action）-> 角色B（2个action），角色B第1个action执行正常，第2个action执行时出现异常退出。


对情况1，执行入口重新执行后，各模块进行反序列化。角色A未观察到属于自己处理的Message，重新执行对应的action。角色B恢复后，观察到一条之前未处理完毕的Message，则在_observe后重新执行对应的react操作，按react策略执行对应2个动作。

对情况2，执行入口重新执行后，各模块进行反序列化。角色A未观察到属于自己处理的Message，不处理。角色B恢复后，_observe到一条之前未完整处理完毕的Message，在react中，知道自己在第2个action执行失败，则直接从第2个action开始执行。



从中断前的Message开始重新执行：

一般来说，Message是不同角色间沟通协作的桥梁，当在Message的执行过程中发生中断后，由于该Message已经被该角色存入环境（Environment）记忆（Memory）中。在进行恢复中，如果直接加载角色全部Memory，该角色的_observe将不会观察到中断时引发当时执行Message，从而不能恢复该Message的继续执行。
因此，为了保证该Message在恢复时能够继续执行，需要在发生中断后，根据_observe到的最新信息，从角色记忆中删除对应的该条Message。

从中断前的Action开始重新执行：

一般来说，Action是一个相对较小的执行模块粒度，当在Action的执行过程中发生中断后，需要知道多个Actions的执行顺序以及当前执行到哪个Action（_rc.state）。当进行恢复时，定位到中断时的Action位置并重新执行该Action。

## Others


- RoadMap: https://github.com/geekan/MetaGPT/blob/main/docs/ROADMAP.md
  - 清晰的长期、短期目标
  - 明确的任务需求，以及版本支持
- FAQ：https://docs.deepwisdom.ai/main/zh/guide/faq.html
  - 文档
  - Tutorial
  - 社区
- RFC：https://docs.deepwisdom.ai/main/zh/rfcs/RFC-116-MetaGPT%E4%BC%98%E5%8C%96%E6%96%B9%E6%A1%88.html
  - 任务初衷
  - 设计过程

## Homework

请梳理MetaGPT的设计思想（也可以挑选其中某一个模块），如果是你会怎么设计？